### Análisis de la red de tiendas - RetailNow

Este notebook implementa el análisis solicitado utilizando Pandas y NumPy sobre los CSV de ventas, inventarios y satisfacción del cliente. El objetivo es apoyar decisiones para optimizar el rendimiento de las tiendas.

Contenido:
- Carga y limpieza de datos
- Exploración y análisis de ventas e ingresos (Pandas)
- Rotación de inventarios y detección de inventario crítico (Pandas)
- Satisfacción del cliente y cruce con desempeño (Pandas)
- Cálculos estadísticos y simulación de proyecciones (NumPy)



In [ ]:
# Imports y configuración
import os
import numpy as np
import pandas as pd

# Configuración de pandas para mejor visualización
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# Rutas absolutas exigidas por el enunciado
SALES_PATH = "/workspace/sales.csv"
INVENTORIES_PATH = "/workspace/inventories.csv"
SATISFACTION_PATH = "/workspace/satisfaction.csv"

# Validación de existencia de archivos (ayuda en depuración)
for path in [SALES_PATH, INVENTORIES_PATH, SATISFACTION_PATH]:
    if not os.path.isabs(path):
        raise ValueError(f"La ruta no es absoluta: {path}")
    if not os.path.exists(path):
        print(f"ADVERTENCIA: No se encuentra el archivo en {path}. "
              f"Asegúrate de que la ruta absoluta apunta a los CSV del proyecto.")


In [ ]:
# 1) Carga de datos desde rutas absolutas
# Nota: el enunciado exige rutas absolutas como /workspace/*.csv
sales_df = pd.read_csv(SALES_PATH)
inventories_df = pd.read_csv(INVENTORIES_PATH)
satisfaction_df = pd.read_csv(SATISFACTION_PATH)

print("Dimensiones iniciales:")
print({
    'sales': sales_df.shape,
    'inventories': inventories_df.shape,
    'satisfaction': satisfaction_df.shape
})

# Vista rápida
display(sales_df.head())
display(inventories_df.head())
display(satisfaction_df.head())


In [ ]:
# 2) Limpieza básica: eliminar filas con valores nulos y normalizar nombres de columnas

def normalizar_columnas(df: pd.DataFrame) -> pd.DataFrame:
    # Estandariza nombres: quita espacios, acentos comunes y reemplaza por guiones bajos
    import unicodedata
    cols = []
    for c in df.columns:
        # Normalización de acentos
        c_norm = ''.join(
            ch for ch in unicodedata.normalize('NFKD', c)
            if not unicodedata.combining(ch)
        )
        c_norm = c_norm.strip().replace(' ', '_')
        cols.append(c_norm)
    df = df.copy()
    df.columns = cols
    return df

sales_df = normalizar_columnas(sales_df).dropna().reset_index(drop=True)
inventories_df = normalizar_columnas(inventories_df).dropna().reset_index(drop=True)
satisfaction_df = normalizar_columnas(satisfaction_df).dropna().reset_index(drop=True)

print("Dimensiones después de limpieza (dropna):")
print({
    'sales': sales_df.shape,
    'inventories': inventories_df.shape,
    'satisfaction': satisfaction_df.shape
})

# Comprobación de estructura esperada
print("Columnas ventas:", list(sales_df.columns))
print("Columnas inventarios:", list(inventories_df.columns))
print("Columnas satisfacción:", list(satisfaction_df.columns))


In [ ]:
# 3) Exploración y análisis de ventas (Pandas)
# Convertimos tipos esenciales
sales_df['Cantidad_Vendida'] = pd.to_numeric(sales_df['Cantidad_Vendida'], errors='coerce')
sales_df['Precio_Unitario'] = pd.to_numeric(sales_df['Precio_Unitario'], errors='coerce')

# Total por registro y métricas derivadas
sales_df['Ingreso'] = sales_df['Cantidad_Vendida'] * sales_df['Precio_Unitario']

# Ventas totales por producto y por tienda
ventas_por_tienda_producto = sales_df.groupby(['ID_Tienda', 'Producto'], as_index=False)['Cantidad_Vendida'].sum()
ventas_por_tienda = sales_df.groupby('ID_Tienda', as_index=False)['Cantidad_Vendida'].sum().rename(columns={'Cantidad_Vendida': 'Total_Ventas'})

# Ingresos totales por tienda
ingresos_por_tienda = sales_df.groupby('ID_Tienda', as_index=False)['Ingreso'].sum().rename(columns={'Ingreso': 'Total_Ingresos'})

# Resumen estadístico de ventas
resumen_ventas = sales_df['Cantidad_Vendida'].describe()

print("Ventas por tienda (cantidades):")
display(ventas_por_tienda)
print("Ingresos por tienda:")
display(ingresos_por_tienda)
print("Ventas por tienda y producto:")
display(ventas_por_tienda_producto)
print("Resumen describe() de Cantidad_Vendida:")
display(resumen_ventas)

# (Opcional) ejemplo si hubiera categorías (no presentes en los datos de ejemplo)
# sales_df['Categoria'] = ...
# promedio_por_tienda_y_categoria = sales_df.groupby(['ID_Tienda', 'Categoria'])['Cantidad_Vendida'].mean().reset_index()


In [ ]:
# 4) Rotación de inventarios y detección de inventario crítico (Pandas)
# Definición: rotación por producto/tienda = ventas_totales_producto_tienda / stock_disponible_producto_tienda
# Para ello, agregamos ventas por tienda y producto, y unimos con inventarios

ventas_tienda_producto = sales_df.groupby(['ID_Tienda', 'Producto'], as_index=False)['Cantidad_Vendida'].sum().rename(columns={'Cantidad_Vendida': 'Ventas_Producto'})

inv = inventories_df.copy()
# Convertimos tipos por seguridad
inv['Stock_Disponible'] = pd.to_numeric(inv['Stock_Disponible'], errors='coerce')

inv_rotacion = inv.merge(ventas_tienda_producto, on=['ID_Tienda', 'Producto'], how='left')
inv_rotacion['Ventas_Producto'] = inv_rotacion['Ventas_Producto'].fillna(0)

# Evitamos división por cero
inv_rotacion['Rotacion'] = np.where(inv_rotacion['Stock_Disponible'] > 0,
                                    inv_rotacion['Ventas_Producto'] / inv_rotacion['Stock_Disponible'],
                                    np.nan)

# Criterio de inventario crítico: porcentaje vendido < 10% del stock disponible
inv_rotacion['Porcentaje_Vendido'] = np.where(inv_rotacion['Stock_Disponible'] > 0,
                                              inv_rotacion['Ventas_Producto'] / inv_rotacion['Stock_Disponible'],
                                              np.nan)
criticos = inv_rotacion[(inv_rotacion['Porcentaje_Vendido'] < 0.10) | (inv_rotacion['Porcentaje_Vendido'].isna())]

print("Inventario con rotación calculada (primeras filas):")
display(inv_rotacion.head())
print("Tiendas/Productos con inventario crítico (<10% vendido):")
display(criticos[['ID_Tienda', 'Producto', 'Stock_Disponible', 'Ventas_Producto', 'Porcentaje_Vendido']])


In [ ]:
# 5) Satisfacción del cliente y cruce con desempeño (Pandas)
# Convertimos tipos y preparamos agregados por tienda
satisfaction_df['Satisfaccion_Promedio'] = pd.to_numeric(satisfaction_df['Satisfaccion_Promedio'], errors='coerce')

# Agregamos ventas e ingresos por tienda (creados previamente)
ventas_por_tienda = sales_df.groupby('ID_Tienda', as_index=False)['Cantidad_Vendida'].sum().rename(columns={'Cantidad_Vendida': 'Total_Ventas'})
ingresos_por_tienda = sales_df.groupby('ID_Tienda', as_index=False)['Ingreso'].sum().rename(columns={'Ingreso': 'Total_Ingresos'})

# Unimos con satisfacción
satisfaccion_tienda = satisfaction_df[['ID_Tienda', 'Satisfaccion_Promedio']].drop_duplicates()
perf_tienda = ventas_por_tienda.merge(ingresos_por_tienda, on='ID_Tienda', how='left')
perf_satisfaccion = perf_tienda.merge(satisfaccion_tienda, on='ID_Tienda', how='left')

# Filtro de baja satisfacción (<60%)
baja_satisfaccion = perf_satisfaccion[perf_satisfaccion['Satisfaccion_Promedio'] < 60]

print("Desempeño por tienda con satisfacción:")
display(perf_satisfaccion)
print("Tiendas con baja satisfacción (<60%):")
display(baja_satisfaccion)

# Breve recomendación heurística basada en datos
recomendaciones = []
for _, row in baja_satisfaccion.iterrows():
    recomendaciones.append({
        'ID_Tienda': row['ID_Tienda'],
        'Recomendacion': (
            "Revisar tiempos de atención, capacitación del personal y surtido; "
            "reforzar promociones en categorías con menor rotación."
        )
    })

print("Recomendaciones para tiendas con baja satisfacción:")
display(pd.DataFrame(recomendaciones))


In [ ]:
# 6) Cálculos con NumPy: mediana y desviación estándar de ventas totales
# Usamos NumPy explícitamente como se solicita
ventas_totales_por_tienda = sales_df.groupby('ID_Tienda', as_index=False)['Cantidad_Vendida'].sum()
ventas_array = ventas_totales_por_tienda['Cantidad_Vendida'].to_numpy()

mediana_np = np.median(ventas_array)
desviacion_std_np = np.std(ventas_array, ddof=0)  # poblacional; usar ddof=1 para muestra

print({
    'ventas_totales_por_tienda': ventas_totales_por_tienda,
    'mediana_numpy': mediana_np,
    'desviacion_estandar_numpy': desviacion_std_np
})


In [ ]:
# 7) Simulación de proyecciones de ventas (NumPy)
# Supongamos que proyectamos ventas para las próximas 12 semanas por tienda
np.random.seed(42)
num_semanas = 12
num_tiendas = ventas_totales_por_tienda.shape[0]

# Punto de partida: vector de ventas actuales por tienda
base = ventas_totales_por_tienda['Cantidad_Vendida'].to_numpy()

# Supongamos una variación normal con media 0 y desviación 10% de la base
variacion_pct = 0.10
# Para cada tienda y semana, generamos una variación porcentual
variaciones = np.random.normal(loc=0.0, scale=variacion_pct, size=(num_semanas, num_tiendas))

# Proyección = base * (1 + variación)
# Aseguramos no-negatividad
proyecciones = np.maximum(base * (1 + variaciones), 0)

# Estadísticas de las proyecciones
promedio_semanal = proyecciones.mean(axis=0)
min_semanal = proyecciones.min(axis=0)
max_semanal = proyecciones.max(axis=0)

print("Proyecciones (primeras 5 semanas):")
display(pd.DataFrame(proyecciones[:5, :], columns=[f"Tienda_{tid}" for tid in ventas_totales_por_tienda['ID_Tienda']]))

print("Resumen por tienda de proyecciones (promedio/min/max):")
resumen_proy = pd.DataFrame({
    'ID_Tienda': ventas_totales_por_tienda['ID_Tienda'],
    'Promedio_Semanal': promedio_semanal,
    'Min_Semanal': min_semanal,
    'Max_Semanal': max_semanal,
})
display(resumen_proy)

# Proyección total semanal agregada
total_semanal = proyecciones.sum(axis=1)
print({
    'total_semanal_promedio': float(total_semanal.mean()),
    'total_semanal_min': float(total_semanal.min()),
    'total_semanal_max': float(total_semanal.max())
})


### Inventario crítico agregado por tienda

Además del análisis por producto, agregamos un indicador a nivel de tienda:
- Calculamos ventas totales y stock total por tienda (sumando productos).
- Definimos inventario crítico de tienda si el porcentaje vendido total < 10%.
Esto permite priorizar tiendas completas con baja salida respecto a su stock total.


In [ ]:
# Agregado: inventario crítico a nivel tienda
# Sumamos stock e igual sumamos ventas (sobre el mismo período)
stock_por_tienda = inventories_df.groupby('ID_Tienda', as_index=False)['Stock_Disponible'].sum().rename(columns={'Stock_Disponible': 'Stock_Total'})
ventas_totales_por_tienda = sales_df.groupby('ID_Tienda', as_index=False)['Cantidad_Vendida'].sum().rename(columns={'Cantidad_Vendida': 'Ventas_Totales'})

a_agg = stock_por_tienda.merge(ventas_totales_por_tienda, on='ID_Tienda', how='left')
a_agg['Ventas_Totales'] = a_agg['Ventas_Totales'].fillna(0)
a_agg['Pct_Vendido_Tienda'] = np.where(a_agg['Stock_Total'] > 0, a_agg['Ventas_Totales'] / a_agg['Stock_Total'], np.nan)

critico_tienda = a_agg[(a_agg['Pct_Vendido_Tienda'] < 0.10) | (a_agg['Pct_Vendido_Tienda'].isna())]

print("Resumen de stock/ventas a nivel tienda:")
display(a_agg)
print("Tiendas críticas por nivel agregado (<10% vendido total):")
display(critico_tienda)
